In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [27]:

dedupOrderFixedfilename = '/nfs/lab/jnewsome/pbmc/cicero_processed_pbmc1_15/refs/merged_loop_file.onlyLoops.dedupe.txt'
# chr10_100027740_100028800 chr10_100069172_100069763


catOrigFileName = '/nfs/lab/jnewsome/pbmc/cicero_processed_pbmc1_15/cicero_output/mergedAllConnsDedup.sort.txt'
# Peak1	Peak2	coaccess
# chr1_10684_9912	chr1_994351_995437	-0.00013886533974112


peakRefFileName = '/nfs/lab/jnewsome/pbmc/cicero_processed_pbmc1_15/refs/peakIDRefFile.bed'
# chr10	100027740	100028800	CicPair_000000001_a



outdir = '/nfs/lab/jnewsome/pbmc/cicero_processed_pbmc1_15/debug/'
pbmc1_12cicFileName = '/nfs/lab/projects/pbmc_snATAC/pipeline/snATAC/cicero/macro/promLabelled_cellMerged/pbmc1to12.Macro.MERGED_ALL.cicero_conns_dedup.promLabelled.txt'
# Peak1_Peak2	Peak1_Chr	Peak1_Start	Peak1_End	Peak2_Chr	Peak2_Start	Peak2_End	Peak1_Promoter	Peak2_Promoter	B_cell_coa	CD4_T_cell_coa	CD8_T_cell_coa	Megakaryocyte_coa	Monocyte_coa	NK_cell_coa	pDC_coa
# chr10_114583064_114583264__chr10_114583482_114583903	chr10	114583064	114583264	chr10	114583482	114583903	.	.	0.990718985139835	0.207702871400228	0.0202850623191396	x	0.103108193932868	-0.0245584803255597	0.111365572442308



peakBedFileOriginalFilename= '/nfs/lab/jnewsome/pbmc/split_lf_mtx/split_peak_bed/mergedBackTogether.bed'


rawConns = '/nfs/lab/jnewsome/pbmc/split_cicero/cat_all_cell_types_raw_conns.justCoords.dedupe.txt'







In [3]:
def orderFix (start, end):
    if int(start) > int(end):
        start, end = end, start
    return start, end


def line_DedupeOrderFixed (line, dict1):
    s = line.split()
    # chr10_100027740_100028800 chr10_100069172_100069763
    # Peak1	Peak2	coaccess
    # chr1_10684_9912	chr1_994351_995437	-0.00013886533974112
    peak1 = s[0]
    peak2 = s[1]
    p1 = peak1.split('_')
    p2 = peak2.split('_')
    p1chrom = p1[0]
    p1start = p1[1]
    p1end = p1[2]
    p2chrom = p2[0]
    p2start = p2[1]
    p2end = p2[2]
    p1start, p1end = orderFix (p1start, p1end)
    p2start, p2end = orderFix (p2start, p2end)
    peak1_key = p1chrom + '_' + p1start + '_' + p1end
    peak2_key = p2chrom + '_' + p2start + '_' + p2end
    peak1_out = p1chrom + '\t' + p1start + '\t' + p1end
    peak2_out = p2chrom + '\t' + p2start + '\t' + p2end
    dict1[peak1_key] = peak1_out
    dict1[peak2_key] = peak2_out
    return dict1
    
    
    
def line_peakRefFile (line, dict1):
    s = line.split()
    # chr10	100027740	100028800	CicPair_000000001_a
    p1chrom = s[0]
    p1start = s[1]
    p1end = s[2]
    p1start, p1end = orderFix (p1start, p1end)
    peak1_key = p1chrom + '_' + p1start + '_' + p1end
    peak1_out = p1chrom + '\t' + p1start + '\t' + p1end
    dict1[peak1_key] = peak1_out
    return dict1

def line_PBMC1_12_CicMergeTable (line, dict1):
    s = line.split()
# Peak1_Peak2	Peak1_Chr	Peak1_Start	Peak1_End	Peak2_Chr	Peak2_Start	Peak2_End	Peak1_Promoter	Peak2_Promoter	B_cell_coa	CD4_T_cell_coa	CD8_T_cell_coa	Megakaryocyte_coa	Monocyte_coa	NK_cell_coa	pDC_coa
# chr10_114583064_114583264__chr10_114583482_114583903	chr10	114583064	114583264	chr10	114583482	114583903	.	.	0.990718985139835	0.207702871400228	0.0202850623191396	x	0.103108193932868	-0.0245584803255597	0.111365572442308
    peak1peak2 = s[0]
    p1p2 = peak1peak2.split('__')
    peak1 = p1p2[0]
    peak2 = p1p2[1]
    p1 = peak1.split('_')
    p2 = peak2.split('_')
    p1chrom = p1[0]
    p1start = p1[1]
    p1end = p1[2]
    p2chrom = p2[0]
    p2start = p2[1]
    p2end = p2[2]
    p1start, p1end = orderFix (p1start, p1end)
    p2start, p2end = orderFix (p2start, p2end)
    peak1_key = p1chrom + '_' + p1start + '_' + p1end
    peak2_key = p2chrom + '_' + p2start + '_' + p2end
    peak1_out = p1chrom + '\t' + p1start + '\t' + p1end
    peak2_out = p2chrom + '\t' + p2start + '\t' + p2end
    dict1[peak1_key] = peak1_out
    dict1[peak2_key] = peak2_out
    return dict1  
    
def readLineToPeakDict (infilename, isDedupeOrderFixed, isOrigCiceroOut, isPeakRefFile, isPBMC1_12_CicMergeTable):
    peakDict = dict()
    
    infile = open(infilename)
    for line in infile:
        line = line.strip()
        if len(line) > 0 and not line.startswith('Peak'):
            
            
            if isDedupeOrderFixed or isOrigCiceroOut:
                peakDict = line_DedupeOrderFixed (line, peakDict)
            if isPeakRefFile:
                peakDict = line_peakRefFile (line, peakDict)
            if isPBMC1_12_CicMergeTable:
                peakDict = line_PBMC1_12_CicMergeTable (line, peakDict)
                
                
    
    infile.close()
    return peakDict
    
    

In [4]:

# orderFixMergedCicFileDict = readLineToPeakDict (dedupOrderFixedfilename, True, False, False, False)



In [5]:
catOrigCicFileDict = readLineToPeakDict (catOrigFileName, False, True, False, False)



In [6]:
peakRefFileDict = readLineToPeakDict (peakRefFileName, False, False, True, False)



In [7]:

pbmc1_12cicFilDict = readLineToPeakDict (pbmc1_12cicFileName, False, False, False, True)


In [8]:
mergedBarePeakDict = readLineToPeakDict (peakBedFileOriginalFilename, False, False, True, False)

In [28]:
rawDict = readLineToPeakDict (rawConns, False, True, False, False)

In [29]:
print('orderFixMergedCicFile = ', len(orderFixMergedCicFileDict.keys()))
print('catOrigCicFileDict = ', len(catOrigCicFileDict.keys()))
print('peakRefFileDict = ', len(peakRefFileDict.keys()))
print('pbmc1_12cicFilDict = ', len(pbmc1_12cicFilDict.keys()))
print('mergedBarePeakDict = ', len(mergedBarePeakDict.keys()))
print('rawDict = ', len(rawDict.keys()))

NameError: name 'orderFixMergedCicFileDict' is not defined

In [14]:
inter_orderFix_catCic = np.intersect1d(list(catOrigCicFileDict.keys()), list(orderFixMergedCicFileDict.keys()))

In [16]:
inter_orderFix_catCicCount = len(inter_orderFix_catCic)
orderFix_ANDNOT_catCicCount = len(orderFixMergedCicFileDict.keys()) - inter_orderFix_catCicCount
catCic_ANDNOT_orderFixCoint = len(catOrigCicFileDict.keys()) - inter_orderFix_catCicCount

print('inter_orderFix_catCicCount = ', inter_orderFix_catCicCount)
print('orderFix_ANDNOT_catCicCount = ', orderFix_ANDNOT_catCicCount)
print('catCic_ANDNOT_orderFixCoint = ', catCic_ANDNOT_orderFixCoint)




inter_orderFix_catCicCount =  238537
orderFix_ANDNOT_catCicCount =  0
catCic_ANDNOT_orderFixCoint =  0


In [18]:
inter_peakRef_catCic = np.intersect1d(list(catOrigCicFileDict.keys()), list(peakRefFileDict.keys()))
inter_peakRef_catCicCount = len(inter_peakRef_catCic)
peakRef_ANDNOT_catCicCount = len(peakRefFileDict.keys()) - inter_peakRef_catCicCount
catCic_ANDNOT_peakRefCount = len(catOrigCicFileDict.keys()) - inter_peakRef_catCicCount

print('inter_peakRef_catCic = ', inter_peakRef_catCicCount)
print('peakRef_ANDNOT_catCicCount = ', peakRef_ANDNOT_catCicCount)
print('catCic_ANDNOT_peakRefCount = ', catCic_ANDNOT_peakRefCount)



inter_peakRef_catCic =  238537
peakRef_ANDNOT_catCicCount =  0
catCic_ANDNOT_peakRefCount =  0


In [19]:
inter_pbmc1_12cic_catCic = np.intersect1d(list(catOrigCicFileDict.keys()), list(pbmc1_12cicFilDict.keys()))
inter_pbmc1_12cic_catCicCount = len(inter_pbmc1_12cic_catCic)
pbmc1_12cic_ANDNOT_catCicCount = len(pbmc1_12cicFilDict.keys()) - inter_pbmc1_12cic_catCicCount
catCic_ANDNOT_pbmc1_12cicCount = len(catOrigCicFileDict.keys()) - inter_pbmc1_12cic_catCicCount

print('inter_pbmc1_12cic_catCicCount = ', inter_pbmc1_12cic_catCicCount)
print('pbmc1_12cic_ANDNOT_catCicCount = ', pbmc1_12cic_ANDNOT_catCicCount)
print('catCic_ANDNOT_pbmc1_12cicCount = ', catCic_ANDNOT_pbmc1_12cicCount)



inter_pbmc1_12cic_catCicCount =  1030
pbmc1_12cic_ANDNOT_catCicCount =  180196
catCic_ANDNOT_pbmc1_12cicCount =  237507


In [11]:
inter_mergedBarePeak_catCic = np.intersect1d(list(catOrigCicFileDict.keys()), list(mergedBarePeakDict.keys()))
inter_mergedBarePeak_catCicCount = len(inter_mergedBarePeak_catCic)
mergedBarePeak_ANDNOT_catCicCount = len(mergedBarePeakDict.keys()) - inter_mergedBarePeak_catCicCount
catCic_ANDNOT_mergedBarePeakCount = len(catOrigCicFileDict.keys()) - inter_mergedBarePeak_catCicCount

print('inter_mergedBarePeak_catCicCount = ', inter_mergedBarePeak_catCicCount)
print('mergedBarePeak_ANDNOT_catCicCount = ', mergedBarePeak_ANDNOT_catCicCount)
print('catCic_ANDNOT_mergedBarePeakCount = ', catCic_ANDNOT_mergedBarePeakCount)


inter_mergedBarePeak_catCicCount =  235657
mergedBarePeak_ANDNOT_catCicCount =  152
catCic_ANDNOT_mergedBarePeakCount =  2880


In [30]:
inter_mergedBarePeak_raw = np.intersect1d(list(rawDict.keys()), list(mergedBarePeakDict.keys()))
inter_mergedBarePeak_rawCount = len(inter_mergedBarePeak_raw)
mergedBarePeak_ANDNOT_rawCount = len(mergedBarePeakDict.keys()) - inter_mergedBarePeak_rawCount
raw_ANDNOT_mergedBarePeakCount = len(rawDict.keys()) - inter_mergedBarePeak_rawCount

print('inter_mergedBarePeak_rawCount = ', inter_mergedBarePeak_rawCount)
print('mergedBarePeak_ANDNOT_rawCount = ', mergedBarePeak_ANDNOT_rawCount)
print('raw_ANDNOT_mergedBarePeakCount = ', raw_ANDNOT_mergedBarePeakCount)





inter_mergedBarePeak_rawCount =  235663
mergedBarePeak_ANDNOT_rawCount =  146
raw_ANDNOT_mergedBarePeakCount =  0


In [34]:
diffList = list()
for key in rawDict.keys():
    s = key.split('_')
    
    diff = int(s[2]) - int(s[1])

    diffList.append(diff)





In [38]:
sum(diffList)/len(diffList)

543.7987083250234

In [12]:
type(inter_mergedBarePeak_catCic)

numpy.ndarray

In [16]:
inter_mergedBarePeak_catCic_dict = {}
for i in list(inter_mergedBarePeak_catCic):
    inter_mergedBarePeak_catCic_dict[i] = ''

In [17]:
crimes_against_chroms = list()
mergedBarePeak_ANDNOT_catCic = list()
for key1 in mergedBarePeakDict:
    if not key1 in inter_mergedBarePeak_catCic_dict:
        mergedBarePeak_ANDNOT_catCic.append(key1)
        s = key1.split('_')
        if not s[0] in crimes_against_chroms:
            crimes_against_chroms.append(s[0])
    
    

In [18]:
crimes_against_chroms2 = list()
catCic_ANDNOT_mergedBarePeak = list()
for key1 in catOrigCicFileDict:
    if not key1 in inter_mergedBarePeak_catCic_dict:
        catCic_ANDNOT_mergedBarePeak.append(key1)
        s = key1.split('_')
        if not s[0] in crimes_against_chroms2:
            crimes_against_chroms2.append(s[0])
    
    

In [24]:
mergedBarePeak_ANDNOT_catCic

lenList_bare_not_cic = list()

for peak in mergedBarePeak_ANDNOT_catCic:
    s = peak.split('_')
    start = s[1]
    end = s[2]
    
    diff = int(end) - int(start)
    if diff < 0:
        print(peak)
    else:
        lenList_bare_not_cic.append(diff)

avg = sum(lenList_bare_not_cic) / len(lenList_bare_not_cic)
print (avg)



286.2894736842105


In [25]:



catCic_ANDNOT_mergedBarePeak

lenList_cic_not_bare = list()

for peak in catCic_ANDNOT_mergedBarePeak:
    s = peak.split('_')
    start = s[1]
    end = s[2]
    
    diff = int(end) - int(start)
    if diff < 0:
        print(peak)
    else:
        lenList_cic_not_bare.append(diff)

avg = sum(lenList_cic_not_bare) / len(lenList_cic_not_bare)
print (avg)





488334.925


In [23]:

dedupOrderFixedOutfilename = outdir + 'orderFix_pbmc1_15_mergedLoopFile.peak.bed' 
catOrigFileOutName = outdir + 'orderFix_pbmc1_15_mergedAllConnsDedupFile.peak.bed' 
peakRefFileOutName = outdir + 'orderFix_pbmc1_15_peakRefFile.peak.bed' 
pbmc1_12cicFileOutName = outdir + 'orderFix_pbmc1_15_pbmc1_12ciceroResults.peak.bed' 


def writeOutfile (outfilename, dict1):
    outfile = open(outfilename, 'w')
    
    for key in dict1.keys():
        outline = dict1[key] + '\t' + key + '\n'
        outfile.write(outline)
    outfile.close()
     
        

writeOutfile (dedupOrderFixedOutfilename, orderFixMergedCicFileDict)
writeOutfile (catOrigFileOutName, catOrigCicFileDict)
writeOutfile (peakRefFileOutName, peakRefFileDict)
writeOutfile (pbmc1_12cicFileOutName, pbmc1_12cicFilDict)